<a href="https://colab.research.google.com/github/evertontech/Resultados-de-Testes-A-B-em-Landing-Pages/blob/main/Resultados_de_Testes_A_B_em_Landing_Pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resultados de Testes A/B em Landing Pages

In [2]:
import pandas as pd
import numpy as np

# Definindo 1000 usuários
N = 1000
np.random.seed(1)

# Criando dados simulados de um teste A/B
data = {
    'user_id': range(1, N + 1),
    'version': np.random.choice(['A', 'B'], N),
    # 1 representa conversão; taxas de conversão ligeiramente diferentes para simular um teste
    'converted': np.random.choice([0, 1], N, p=[0.89, 0.11])
}

df_ab = pd.DataFrame(data)

# Exibe o início do novo DataFrame
print(df_ab.head())


   user_id version  converted
0        1       B          0
1        2       B          0
2        3       A          0
3        4       A          0
4        5       B          0


**Calculando a Taxa de Conversão**

In [4]:
df_ab.groupby('version')['converted'].mean()

,converted
version,
A,0.111111
B,0.102510


**Versão B performou pior que a Versão A**

**Análise Diagnóstica**
**Essa diferença de 0.86 ponto percentual é real e causada pela nossa mudança, ou é apenas ruído (sorte/azar) em nossa amostra de 1000 usuários?**

**Estatística Inferencial e calcular a Significância Estatística (o famoso P-valor)**

In [8]:
N_observacoes = df_ab.groupby('version')['converted'].count()
print(f"Total de Observações por Versão:\n{N_observacoes}")

Total de Observações por Versão:
version
A    522
B    478
Name: converted, dtype: int64


In [11]:
totalDeSucessos = df_ab.groupby('version')['converted'].sum()
print(f"Total de Sucessos: \n {totalDeSucessos}")

Total de Sucessos: 
 version
A    58
B    49
Name: converted, dtype: int64


**Como a Versão B teve uma conversão mais baixa, precisamos saber: "Essa queda é real (significativa) ou é aleatória?"**

Para responder, vamos rodar o **Teste Z para Duas Proporções**, que nos dará o **P-valor.**

**P-valor:** A probabilidade de observarmos uma diferença de 0.86pp ou mais, assumindo que as duas páginas são realmente idênticas. Se o P-valor for muito baixo (geralmente menor que **0.05**), a diferença é considerada real.

In [12]:
from statsmodels.stats.proportion import proportions_ztest

# 1. Definindo os inputs K (sucessos) e N (observações)
count = [58, 49] # Sucessos A e Sucessos B
nobs = [522, 478] # Observações A e Observações B

# 2. Rodando o Teste Z
z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

# 3. Imprimindo os resultados
print(f"Estatística Z: {z_stat:.4f}")
print(f"P-valor: {p_value:.4f}")


Estatística Z: 0.4395
P-valor: 0.6603


 diagnóstico final é: A queda de $11.11\%$ para $10.25\%$ na Versão B foi devida a flutuações aleatórias da amostra, e não uma mudança real causada pelo seu design.

O P-valor foi de $0.6603$ (ou $66.03\%$).Regra: Em negócios, geralmente só aceitamos que uma diferença é real se o P-valor for menor que $0.05$ (ou $5\%$).Diagnóstico: Como $0.6603$ é muito maior que $0.05$, não podemos rejeitar a Hipótese Nula.